### 第3章: 正規表現
Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

1行に1記事の情報がJSON形式で格納される 各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される ファイル全体はgzipで圧縮される 以下の処理を行うプログラムを作成せよ．

In [4]:
!mkdir -p data/chapter3 && wget -P data/chapter3/ https://nlp100.github.io/data/jawiki-country.json.gz
!gunzip data/chapter3/jawiki-country.json.gz

--2023-11-23 10:44:44--  https://nlp100.github.io/data/jawiki-country.json.gz
Resolving nlp100.github.io (nlp100.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5068362 (4.8M) [application/gzip]
Saving to: ‘data/chapter3/jawiki-country.json.gz’

jawiki-country.json 100%[===================>]   4.83M  2.76MB/s    in 1.8s    

2023-11-23 10:44:46 (2.76 MB/s) - ‘data/chapter3/jawiki-country.json.gz’ saved [5068362/5068362]



## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [10]:
!cat data/chapter3/jawiki-country.json | jq 'select(.title=="イギリス")'  | jq '.text' | sed -e 's/\\n/\n/g' > data/chapter3/20.out

### 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [11]:
!cat data/chapter3/20.out | grep '\[Category'

[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]"


### 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [12]:
!cat data/chapter3/20.out | grep '\[\[Category' | sed -E 's/\[\[Category:(.*)]].?$/\1/' | sed -e 's/[\|\*]//g'

イギリス
イギリス連邦加盟国
英連邦王国
G8加盟国
欧州連合加盟国元
海洋国家
現存する君主国
島国
1801年に成立した国家・領域


### 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [50]:
import re

with open('data/chapter3/20.out', 'r') as f:
    string = ''.join(f.readlines())
    pattern = r'(\={2,})(.*?)\={2,}'
    n_labels = re.findall(pattern, string)
    result = '\n'.join(["{0}:{1}".format(x[1], len(x[0])-1) for x in n_labels])
    print(result)


国名:1
歴史:1
地理:1
主要都市:2
気候:2
政治:1
元首:2
法:2
内政:2
地方行政区分:2
外交・軍事:2
経済:1
鉱業:2
農業:2
貿易:2
不動産:2
エネルギー政策:2
通貨:2
企業:2
通信:3
交通:1
道路:2
鉄道:2
海運:2
航空:2
科学技術:1
国民:1
言語:2
宗教:2
婚姻:2
移住:2
教育:2
医療:2
文化:1
食文化:2
文学:2
哲学:2
音楽:2
ポピュラー音楽:3
映画:2
コメディ:2
国花:2
世界遺産:2
祝祭日:2
スポーツ:2
サッカー:3
クリケット:3
競馬:3
モータースポーツ:3
野球:3
 カーリング :3
 自転車競技 :3
脚注:1
関連項目:1
外部リンク:1


### 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．